In [1]:
import pandas as pd
import numpy as np
import pickle

ED: DEMOGRAPHICS

In [ ]:
AGES = [0, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110]
AGE_LABELS = [f'Age/{AGES[i]}-{AGES[i+1]}' for i in range(0, len(AGES)-1)]
def age_str(x):
    ind = np.searchsorted(AGES, x, 'right')
    return AGE_LABELS[ind-1]

In [ ]:
def race_str(x):
    if 'ASIAN' in x:
        return 'RACE/ASIAN'
    elif 'BLACK' in x:
        return 'RACE/BLACK'
    elif 'WHITE' in x:
        return 'RACE/WHITE'
    elif 'HISPANIC' in x:
        return 'RACE/HISPANIC'
    elif 'BLACK' in x:
        return 'RACE/BLACK'
    elif 'NATIVE' in x:
        return 'RACE/AMERICAN'
    elif x == 'PORTUGUESE': return 'RACE/PORTUGUESE'
    elif x == 'SOUTH AMERICAN': return 'RACE/SOUTH'
    else:
        return 'RACE/UNKNOWN'

In [2]:
demo_df = pd.read_csv('data/transfer/demo.csv')

In [33]:
demo_df = demo_df.drop(columns=['hadm_id', 'anchor_age', 'anchor_year', 'ethnicity'])

In [ ]:
demo_df['race_str'] = demo_df['race'].apply(race_str)
demo_df['gender_str'] = 'Gender/' + demo_df['gender']
demo_df['age_str'] = demo_df['age'].apply(lambda x: age_str(x) if not np.isnan(x) else None)

In [6]:
demo_df.to_csv('data/transfer/demo.csv', index=False)

LABEL

In [3]:
df = pd.read_csv('data/transfer/label.csv', usecols=['subject_id', 'stay_id', 'intime', 'outcome_icu_transfer_12h', 'Time'])

In [5]:
df = df.rename(columns={'outcome_icu_transfer_12h': 'Label'})

In [7]:
df.to_csv('data/transfer/label.csv', index=False)

MED

In [16]:
df = pd.read_csv('data/transfer/med.csv', index_col=0)

In [17]:
df = df[['subject_id', 'stay_id', 'charttime', 'group']]

In [20]:
df['eventval'] = 'MED/' + df['group'].astype(str)

In [22]:
df.to_csv('data/transfer/med.csv', index=False)

VITALS

In [ ]:
def bucket_eventval(event, val, d):
    buckets = d[event]
    if type(buckets) == str: 
        return buckets
    ind = np.searchsorted(buckets, val, side='right')
    if ind == len(buckets):
        eventval = f"{event}|{buckets[ind-1]}-"
    else:
        eventval = f"{event}|{buckets[ind-1]}-{buckets[ind]}"
    return eventval

In [ ]:
def bucket_ind(event, val, d):
    buckets = d[event]
    if type(buckets) == str: 
        return 0
    ind = np.searchsorted(buckets, val, side='right')
    return ind

In [2]:
df = pd.read_csv('data/transfer/numerics.csv')

In [3]:
df_pivot = None
for c in df.columns:
    if c == 'stay_id' or c == 'charttime': continue
    df_c = df[['stay_id', 'charttime'] + [c]]
    df_c = df_c.rename(columns={c:'value'})
    df_c['event'] = c
    if df_pivot is None: df_pivot = df_c
    else: df_pivot = pd.concat([df_pivot, df_c])

In [4]:
df_pivot = df_pivot.dropna(subset=['value'])

In [ ]:
with open('data/buckets/ed_buckets.pkl', 'rb') as file:
    ed_buckets = pickle.load(file)

In [8]:
df_pivot['eventval'] = df_pivot.apply(lambda x: bucket_eventval(x['event'], x['value'], ed_buckets), axis=1)

In [10]:
df_pivot.to_csv('data/transfer/numerics.csv', index=False)